# EXTERNAL FACTORS WITH THE ORIGINAL DATASET

In [10]:
import pandas as pd
import requests

hospital_data = pd.read_csv("/finalproject/collection_process/hospital_admissions.csv") 

In [11]:

hospital_data['ResidentDate'] = pd.to_datetime(
    hospital_data.rename(columns={
        'ResidentDate_year': 'year',
        'ResidentDate_month': 'month',
        'ResidentDate_day': 'day'
    })[['year', 'month', 'day']]
)

# Check if the date looks correct
print(hospital_data[['ResidentDate_year', 'ResidentDate_month', 'ResidentDate_day', 'ResidentDate']].head())


   ResidentDate_year  ResidentDate_month  ResidentDate_day ResidentDate
0               2017                   3                21   2017-03-21
1               2017                   3                21   2017-03-21
2               2017                   3                21   2017-03-21
3               2017                   3                21   2017-03-21
4               2017                   3                21   2017-03-21


In [12]:
import pandas as pd
import requests

# CONFIG
LAT = 13.0827
LON = 80.2707
START_DATE = "2017-03-21"
END_DATE = "2022-03-20"
API_KEY = '8TEpIGk3AuzzHb0vNIBlWsfq0aeSp6H9'  # Calendarific API Key

# Fetch Weather from Open-Meteo
def fetch_weather(start_date, end_date):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        'latitude': LAT,
        'longitude': LON,
        'start_date': start_date,
        'end_date': end_date,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'timezone': 'Asia/Kolkata'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return pd.DataFrame({
        'ResidentDate': pd.to_datetime(data['daily']['time']).date,
        'temperature_max': data['daily']['temperature_2m_max'],
        'temperature_min': data['daily']['temperature_2m_min'],
        'precipitation': data['daily']['precipitation_sum']
    })

# Fetch Holidays using Calendarific
def fetch_holidays(year, api_key):
    url = "https://calendarific.com/api/v2/holidays"
    params = {
        'api_key': api_key,
        'country': 'IN',
        'year': year,
        'location': 'Tamil Nadu',
    }
    response = requests.get(url, params=params).json()
    holidays = []
    for item in response['response']['holidays']:
        holidays.append({
            'ResidentDate': pd.to_datetime(item['date']['iso']).date(),
            'event_type': 'Holiday'
        })
    return pd.DataFrame(holidays)

# Covid Waves — returns a dataframe
def generate_covid_waves():
    covid_periods = [
        ('2020-03-20', '2020-06-30'),
        ('2021-04-15', '2021-06-15'),
        ('2022-01-01', '2022-03-01')
    ]
    covid_list = []
    for start, end in covid_periods:
        date_range = pd.date_range(start, end)
        covid_list.extend([{'ResidentDate': d.date(), 'pandemic': 'Covid_Wave'} for d in date_range])
    return pd.DataFrame(covid_list)

# Disaster Events — returns a dataframe
def fetch_disaster_events():
    df = pd.read_csv('/finalproject/collection_process/disaster_data_chennai_2016_2022_gemini.csv')
    # Specify the correct format for the date conversion
    df['ResidentDate'] = pd.to_datetime(df['ResidentDate'], format='%d/%m/%Y').dt.date
    df = df[['ResidentDate', 'event_type']]
    df = df.rename(columns={'event_type': 'disaster'})
    return df

# Merge all external features
def merge_all_events(start_date, end_date, api_key):
    weather = fetch_weather(start_date, end_date)

    # Fetch holidays
    holiday_frames = [fetch_holidays(year, api_key) for year in range(
        pd.to_datetime(start_date).year, pd.to_datetime(end_date).year + 1)]
    holidays = pd.concat(holiday_frames, ignore_index=True)

    # Fetch pandemic and disaster data
    covid = generate_covid_waves()
    disasters = fetch_disaster_events()

    # Sundays
    date_range = pd.DataFrame({'ResidentDate': pd.date_range(start_date, end_date)})
    date_range['weekday'] = date_range['ResidentDate'].dt.weekday
    sundays = date_range[date_range['weekday'] == 6][['ResidentDate']].copy()
    sundays['event_type'] = 'Sunday'
    sundays['ResidentDate'] = sundays['ResidentDate'].dt.date

    # Combine general event types (Holiday + Sunday)
    general_events = pd.concat([holidays, sundays], ignore_index=True)
    general_events = general_events.drop_duplicates(subset=['ResidentDate'])

    # Merge all on ResidentDate
    merged = pd.merge(weather, general_events, on='ResidentDate', how='left')
    merged = pd.merge(merged, covid, on='ResidentDate', how='left')
    merged = pd.merge(merged, disasters, on='ResidentDate', how='left')

    # Fill missing
    merged['event_type'] = merged['event_type'].fillna('None')
    merged['pandemic'] = merged['pandemic'].fillna('None')
    merged['disaster'] = merged['disaster'].fillna('None')

    return merged

# Merge with Hospital Data
def merge_with_hospital_data(hospital_data, external_factors):
    hospital_data = hospital_data.rename(columns={
        'ResidentDate_year': 'year',
        'ResidentDate_month': 'month',
        'ResidentDate_day': 'day'
    })

    hospital_data['ResidentDate'] = pd.to_datetime(
        hospital_data[['year', 'month', 'day']]
    ).dt.date

    hospital_data = hospital_data.rename(columns={
        'year': 'ResidentDate_year',
        'month': 'ResidentDate_month',
        'day': 'ResidentDate_day'
    })

    merged_data = pd.merge(hospital_data, external_factors, on='ResidentDate', how='left')
    return merged_data

# Usage Example
hospital_data = pd.read_csv('/finalproject/collection_process/hospital_admissions.csv')

# Fetch enriched external factors
external_factors = merge_all_events(START_DATE, END_DATE, API_KEY)

# Merge hospital data with external factors
merged_hospital_data = merge_with_hospital_data(hospital_data, external_factors)

# Save to CSV
merged_hospital_data.to_csv('/finalproject/collection_process/hospital_data_with_external_factors.csv', index=False)

print("✅ Saved: 'hospital_data_with_external_factors.csv'")


✅ Saved: 'hospital_data_with_external_factors.csv'
